In [1]:
import pandas as pd
import numpy as np
import dask.bag as db
import dask.dataframe as frame
import re

In [2]:
lines = db.read_text(r"C:\Users\hwe334\OneDrive - Anadarko\Python Scripts\data\example_2.txt").str.strip('-').str.split('\n')

In [3]:
#convert dask bag to dask dataframe
df = lines.to_dataframe().drop(1,axis=1)

In [4]:
#name columns and create ref lines
df.columns = ['line']
df['line_num'] = df.index + 1

In [5]:
#split lines in to Odds and Evens 
odds = df[df.line_num %  2 == 0].reset_index().drop('index', axis=1)
evens = df[df.line_num %  2 == 1].reset_index().drop('index',axis=1)

In [6]:
#odds = odds.where(odds['line'] != '').dropna().reset_index(drop=True)
#evens = evens.where(evens['line'] != '').dropna().reset_index(drop=True)

In [7]:
#get headers from odds and evens DFs
head1 = odds[1:1]['line'].values.compute()[0].split('|')
head2 = evens[2:2]['line'].values.compute()[0].split('|')

In [8]:
# Combine Headers from odds and evens dataframes
header = head1 + head2

In [9]:
# merge odds and even dd to single dd
ddf = evens[3:].join(odds[1:],lsuffix="main",rsuffix="wrapped" )
ddf = ddf.drop(['line_nummain','line_numwrapped'], axis=1)

In [10]:
# join lines from both dd
ddf['wholeline'] = ddf['linemain'].astype(str) + ddf['linewrapped'].astype(str)
df = ddf['wholeline']

In [11]:
# Save File to Exel
SR = df.compute()
df = pd.DataFrame([ x.split('|') for x in SR.tolist() ], columns = header).drop('',axis=1)
df.to_excel('complex_text.xlsx')